In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/cve-dataset-with-embeddings-using-bert/CVE_embeddings_bert.csv')

In [ ]:
df.shape

In [ ]:
df.columns[:87]

In [ ]:
df['cve.metrics.cvssMetricV2.cvssData.availabilityImpact']

In [ ]:
dfi = pd.concat([df.iloc[:, 87:], df[['cve.metrics.cvssMetricV2.cvssData.availabilityImpact']]], axis=1)

In [ ]:
dfi

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
import pickle

# Assuming dfi is your DataFrame
X = dfi.iloc[:, :-1]  # Features
y = dfi.iloc[:, -1]   # Target labels

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)

    # Since R² is not typically used for classification, we will print it but understand it may not be meaningful
    r2 = r2_score(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}")
    print(f"{model_name} R² Score: {r2:.4f}\n")

    # Save the model as a pickle file
    with open(f'{model_name.lower()}_model.pkl', 'wb') as f:
        pickle.dump(model, f)

# Initialize models
models = {
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the label encoder as well, in case you need to decode predictions later
with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
import pickle

# Assuming dfi is your DataFrame
X = dfi.iloc[:, :-1]  # Features
y = dfi.iloc[:, -1]   # Target labels

# Encode categorical labels into numeric values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Function to train and evaluate a model
def train_and_evaluate(model, model_name):
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)

    # Calculate metrics
    accuracy = accuracy_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    mae = mean_absolute_error(y_test, predictions)

    print(f"{model_name} Accuracy: {accuracy:.4f}")
    print(f"{model_name} Mean Squared Error: {mse:.4f}")
    print(f"{model_name} Mean Absolute Error: {mae:.4f}\n")

    # Save the model as a pickle file
    with open(f'{model_name.lower()}_model.pkl', 'wb') as f:
        pickle.dump(model, f)

# Initialize models
models = {
    'Support Vector Machine': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Train and evaluate each model
for name, model in models.items():
    train_and_evaluate(model, name)

# Save the label encoder as well, in case you need to decode predictions later
with open('label_encoder.pkl', 'wb') as le_file:
    pickle.dump(label_encoder, le_file)
